In [1]:
import os
import time
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings

d:\codespace\ML\NYD\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()
# Load the GROQ and OpenAI API keys
groq_api_key = os.getenv('GROQ_API_KEY')
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

print("RAG Doc Assistant Using Llama3")

RAG Doc Assistant Using Llama3


In [3]:
# Initialize the LLM
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="Llama3-8b-8192"
)

In [ ]:
# prompt = ChatPromptTemplate.from_template(
#     """
#     you are a bhagwatgita chatbot , you will have to provide the responses based on the bhagwatgita document provided by the user
#     and the question asked about it you have to provide responses mostly based on the document not in general.
#     <context>
#     {context}
#     <context>
#     Questions:{input}
#     """
# )

In [4]:
# Create the prompt template
prompt = ChatPromptTemplate.from_template(
    """
    You are a document assistant that helps users find information in a context.
    Please provide the most accurate response based on the context and inputs.
    Only give information that is in the context, not general information.
    <context>
    {context}
    <context>
    Questions: {input}
    """
)


In [5]:
# Function to process the uploaded PDF and create vector embeddings
def vector_embedding(file_path):
    print("Processing file:", file_path)
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    loader = PyPDFLoader(file_path)  # Load PDF
    docs = loader.load()  # Document Loading
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)  # Chunk Creation
    final_documents = text_splitter.split_documents(docs[:20])  # Splitting
    vectors = FAISS.from_documents(final_documents, embeddings)  # Vector embeddings
    print("Vector Store DB is ready.")
    return vectors

In [ ]:
# Input the PDF file path
file_path = input("Enter the path to the PDF file: ").strip()
if not os.path.isfile(file_path):
    print("File not found. Please provide a valid file path.")
else:
    vectors = vector_embedding(file_path)

    # Start the question loop
    while True:
        question = input("Enter your question (or type 'exit' to quit): ").strip()
        if question.lower() == 'exit':
            break
        
        # Create the retrieval chain and get the response
        document_chain = create_stuff_documents_chain(llm, prompt)
        retriever = vectors.as_retriever()
        retrieval_chain = create_retrieval_chain(retriever, document_chain)
        
        start = time.process_time()
        response = retrieval_chain.invoke({'input': question})
        print("Response time:", time.process_time() - start)
        print("Answer:", response['answer'])
        
        # Display similar document chunks
        print("\nDocument Similarity Search Results:")
        for i, doc in enumerate(response["context"]):
            print(f"Chunk {i + 1}:\n{doc.page_content}")
            print("--------------------------------")

Processing file: C:\Users\abhil\Downloads\NYDProgess.pdf
Vector Store DB is ready.
Response time: 0.015625
Answer: Based on the context, the progress report is for the NYD Hackathon Week 1. Here is the summary:

* Setuped Web Search Retrieval on Microsoft Azure
* Implemented the RAG functionality as intended properly
* Setuped the base LLMs in local environments
* Tried to fine-tune Legacy Model GPT2, but it did not meet expectations
* Planned to use different models, such as llama3, Gemma2, Gemini API, and Cohere API
* Used llama3 with ollama locally for the langchain pipeline building

No major achievements or milestones mentioned, but the report highlights the steps taken to set up the necessary infrastructure and explore different models for the task at hand.

Document Similarity Search Results:
Chunk 1:
 Tried Out Various RAG templates for the starƟng.  
 
 
 Setuped Web Search Retrival On MicrosoŌ Azure. 
 
 
 
 Working  On ImplemenƟng The RAG funcƟonality As Intended Properly